<a href="https://colab.research.google.com/github/ssannkkallpp/covidsimulator/blob/master/Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COVID-19 Simulation
Refer to https://hackmd.io/@sudhanshu2/BkpvZp-oU for notes.

In [0]:
import networkx as nx
import matplotlib.pyplot as plt
from random import choice 
import random

from bokeh.io import output_file, show, output_notebook
from bokeh.models import (BoxSelectTool, Circle, EdgesAndLinkedNodes, HoverTool,
                          MultiLine, NodesAndLinkedEdges, Plot, Range1d, TapTool, 
                          BoxZoomTool, ResetTool, ColumnDataSource, Legend, LegendItem, )
from bokeh.plotting import figure, show
from bokeh.palettes import Spectral4
from bokeh.models.graphs import from_networkx

output_notebook()

In [0]:
# Intialize Values

population = 10**3
infected = 100
susceptible = 0
recovered = 0
probability = 0.02
total_infected = 100 # we initially set the total number of infected people to the seed value but it is dynamically updated as 

# Age Distribution for 0-14, 15-24, 25-44, 45-64 and 65+ in India
age_distribution = [35.3, 18.4, 27.6, 13.5, 4.8]

In [0]:
# Initialize Graph 

G = nx.fast_gnp_random_graph(population, probability)
pos = nx.random_layout(G)

for (u, v) in G.edges():
  G.edges[u,v]['weight'] = random.randint(0,100)

In [0]:
# Color Nodes

colour_list = {"AD" : 0, "T" : 0, "YA" : 0, "MA" : 0, "O" : 0}
node_attrs = {}

for node in G.nodes:
  if ((100.0 * colour_list["AD"]) / population) < age_distribution[0]:
    node_attrs[node] = {'age': "AD", 'node_color': "#81C784"}
    colour_list["AD"] += 1
  elif ((100.0 * colour_list["T"]) / population) < age_distribution[1]:
    node_attrs[node] = {'age': "T", 'node_color': "#4FC3F7"}
    colour_list["T"] += 1
  elif ((100.0 * colour_list["YA"]) / population) < age_distribution[2]:
    node_attrs[node] = {'age': "YA", 'node_color': "#FFF176"}
    colour_list["YA"] += 1
  elif ((100.0 * colour_list["MA"]) / population) < age_distribution[3]:
    node_attrs[node] = {'age': "MA", 'node_color': "#FFB74D"}
    colour_list["MA"] += 1
  else:
    node_attrs[node] = {'age': "O", 'node_color': "#E57373"}
    colour_list["O"] += 1

nx.set_node_attributes(G, node_attrs)

In [5]:
# Bokeh plot graph

plot = Plot(plot_width=600, plot_height=600, x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))

graph_renderer = from_networkx(G, nx.spring_layout, scale=1, center=(0, 0))
plot.add_tools(BoxZoomTool(), ResetTool(), TapTool(), BoxSelectTool())

graph_renderer.node_renderer.glyph = Circle(size=15, fill_color="node_color", line_color="#616161")
graph_renderer.node_renderer.selection_glyph = Circle(size=15, fill_color="node_color", line_color="#616161")
graph_renderer.node_renderer.hover_glyph = Circle(size=15, fill_color="node_color", line_color="#616161")

graph_renderer.edge_renderer.glyph = MultiLine(line_color="#BDBDBD", line_alpha=0.8, line_width=1)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color="#616161", line_width=1)
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color="#212121", line_width=1)

graph_renderer.selection_policy = NodesAndLinkedEdges()
graph_renderer.inspection_policy = EdgesAndLinkedNodes()

plot.renderers.append(graph_renderer)

show(plot)

In [62]:
# Simulate spread

current_time = 1
currently_infected = set()
not_infected = set()

list_time = list()
list_alive = list()
list_dead = list()
list_susceptible = list()
list_infected = list()
list_recovered = list()

current_dead = 0
current_recovered = 0
current_infected = infected

label = {}	
count = 0
for u in G:
  if (random.randint(0,1) == 1) and count < infected:
    label[u] = ("I", current_time)
    count = count + 1
    currently_infected.add(u)
  else:
    label[u] = ("S", current_time)
    
count_no_spread_days = 0

current_susceptible = population - infected
list_time.append(1)
list_alive.append(population - current_dead)
list_dead.append(0)
list_susceptible.append(current_susceptible)
list_infected.append(infected)
list_recovered.append(0)

while count_no_spread_days < 30:
  current_time += 1
  new_infections = set()
  for u in currently_infected:
    if label[u][0] == 'I':
      if (current_time - label[u][1]) > 27:
        if random.randint(0, 10) > 1:
          label[u] = ("R", current_time)
          current_recovered += 1
          current_infected -= 1
        else:
          label[u] = ("D", current_time)
          current_dead += 1
          current_infected -= 1
      else:
        for v in list(G.neighbors(u)):
          if label[v][0] == 'S' and G.edges[u,v]['weight'] < 6:
            label[v] = ("I", current_time)
            new_infections.add(v)
            current_infected += 1
  currently_infected = currently_infected.union(new_infections)
  if len(new_infections) == 0:
    count_no_spread_days += 1
  current_susceptible = population - current_infected - current_dead - current_recovered
  list_time.append(current_time)
  list_alive.append(population - current_dead)
  list_dead.append(current_dead)
  list_susceptible.append(current_susceptible)
  list_infected.append(current_infected)
  list_recovered.append(current_recovered)

print(list_time)
print(list_alive)
print(list_dead)
print(list_susceptible)
print(list_infected)
print(list_recovered) 

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]
[1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 979, 958, 941, 933, 924, 919, 916, 914, 913, 911, 911, 910, 910, 910]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 21, 42, 59, 67, 76, 81, 84, 86, 87, 89, 89, 90, 90, 90]
[900, 802, 718, 664, 621, 592, 571, 557, 548, 544, 542, 540, 540, 540, 540, 540, 540, 540, 540, 540, 540, 540, 540, 540, 540, 540, 540, 540, 540, 540, 540, 540, 540, 540, 540, 540, 540, 540, 540, 540, 540, 540]
[100, 198, 282, 336, 379, 408, 429, 443, 452, 456, 458, 460, 460, 460, 460, 460, 460, 460, 460, 460, 460, 460, 460, 460, 460, 460, 460, 460, 360, 262, 178, 124, 81, 52, 31, 17, 8, 4, 2, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [63]:
# Plot graph

xs = [list_time, list_time, list_time, list_time, list_time]
ys = [list_alive, list_dead, list_susceptible, list_infected, list_recovered]
line_colors = ["#4CAF50", "#F44336", "#FF9800", "#03A9F4", "#673AB7"]

p = figure(plot_width=800, plot_height=500)
r = p.multi_line(xs, ys, color=line_colors, line_alpha=0.6, line_width=2)

legend = Legend(items=[
    LegendItem(label="alive", renderers=[r], index=0),
    LegendItem(label="dead", renderers=[r], index=1),
    LegendItem(label="susceptible", renderers=[r], index=2),
    LegendItem(label="infected", renderers=[r], index=3),
    LegendItem(label="recovered", renderers=[r], index=4),
])

p.add_layout(legend)
show(p)

## In-development
Running these blocks might affect the rest of the code, run them with caution.

In [0]:
def spread(G, size, labels):
	total_infected = 100
	tround = 1 # this variable marks the round of transmission 
	while True:
		tround += 1
		for v in list(G.nodes):

			if labels[v][0] == "R" or labels[v][0] == "D":
				continue
			# ignore the nodes that are either dead or recovered

			elif labels[v][0] == "S":
				neighbors = list(G.neighbors(v))
				for u in neighbors:  # for nodes that are still susceptible
					if labels[u][0] == "I":
						if labels[u][1] in range(2, 10): 
							if G[v][u]['weight'] > 0.3:
								labels[v] = ("I", tround)
								total_infected += 1
						else:
							if G[v][u]['weight'] > 0.6:
								labels[v] = ("I", tround)
								total_infected += 1

			else:
				# in this case the node is already infected or "I"
				if (tround - labels[v][1]) > 27:
				# if the person has been infected for more than an average of 4 weeks
				# then they either die or recovers.
				# a day here maps to one round of transmission
					if random.randint(0, 100) < 40:
						labels[v] = ("R", 0)
					else:
						labels[v] = ("D", 0)
				continue

		print(total_infected)


TabError: ignored

In [0]:
def create_graph(size, p):
	G = networkx.erdos_renyi_graph(size, p)
	positions = networkx.spring_layout(G)
	map = colour_graph(G, size)
	count = 0
	labels = {}	
	for node in G:
		if (random.randint(0,1) == 1) and count < infected:
			labels[node] = ("I", 1)
			count = count + 1
		else:
			labels[node] = ("S", 0)
	 
	for (u, v) in G.edges():
		num = random.randint(1,100)
		G[u][v]['weight'] = (1.0/num) # edge weights are transition probabilities 

		# initially we give the infected label randomly to some percentage of the population
		# and everyone else is initially susceptible 
	spread(G, size, labels) # call spread function to compute basic simulation on 
	networkx.draw(G, node_color=map, with_labels=True)
  networkx.draw_network_labels(G, positions, labels, font_size=16,font_color='black'))